In [ ]:
from google.colab import drive
drive.mount('/content/drive')
cd='/content/drive/My Drive/Colab Notebooks/'
%cd /content/drive/My Drive/Colab Notebooks/
import numpy as np
import os, cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

import keras 
from keras.models import Sequential
from keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Colab Notebooks


In [ ]:
data_path = '/content/drive/MyDrive/Colab Notebooks/CK+48'
data_dir_list = os.listdir(data_path)
img_data_list=[]
labels_list=[]
class_value=0 #i is the label 
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    for img in img_list:
        # Using 0 to read image in grayscale mode
        input_img=cv2.imread(data_path + '/'+ dataset + '/'+ img, 0 )
        #input_img=cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        #input_img_resize=cv2.resize(input_img,(48,48))
        img_data_list.append(input_img)
        labels_list.append(class_value)
    class_value+=1
        
img_data_list = np.array(img_data_list)
img_data_list = img_data_list.astype('float32')
img_data_list = img_data_list/np.amax(img_data_list)#normalize data by dividing each pixel value by the maximum
img_data_list.shape #The dataset contains 750 images, each image is a 2D shape 48x48

Loaded the images of dataset-surprise

Loaded the images of dataset-anger

Loaded the images of dataset-sadness

Loaded the images of dataset-fear

Loaded the images of dataset-happy



(750, 48, 48)

In [ ]:
labels = np.array(labels_list)
labels = labels.astype('int32')
print(labels.shape)

(750,)


In [ ]:
#Reshape images to 1D
img_data_list=np.reshape(img_data_list, newshape=(img_data_list.shape[0],-1))
img_data_list.shape

(750, 2304)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(img_data_list, labels, test_size=0.2)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2)

y_train = to_categorical(y_train, 5)
y_valid = to_categorical(y_valid, 5)
y_test = to_categorical(y_test, 5)

In [ ]:
model = Sequential()
model.add(Input(2304)) #reshaped input from 2 D 48x48 to 1D (48*48)=2304
model.add(Dense(1024, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation = 'softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              2360320   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               131200    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 645       
Total params: 2,492,165
Trainable params: 2,492,165
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adam())
mlp_training = model.fit(x_train, y_train, batch_size=32, epochs=50, verbose=1, validation_data=(x_valid, y_valid))

Epoch 1/50
15/15 [==============================] - 1s 22ms/step - loss: 1.0822 - accuracy: 0.5417 - val_loss: 0.8446 - val_accuracy: 0.7167
Epoch 2/50
15/15 [==============================] - 0s 14ms/step - loss: 0.9748 - accuracy: 0.5479 - val_loss: 0.7879 - val_accuracy: 0.6417
Epoch 3/50
15/15 [==============================] - 0s 13ms/step - loss: 0.9035 - accuracy: 0.6187 - val_loss: 0.7521 - val_accuracy: 0.7333
Epoch 4/50
15/15 [==============================] - 0s 13ms/step - loss: 0.8484 - accuracy: 0.6104 - val_loss: 0.6929 - val_accuracy: 0.6333
Epoch 5/50
15/15 [==============================] - 0s 13ms/step - loss: 0.8390 - accuracy: 0.6333 - val_loss: 0.6117 - val_accuracy: 0.7833
Epoch 6/50
15/15 [==============================] - 0s 13ms/step - loss: 0.7683 - accuracy: 0.6812 - val_loss: 0.6206 - val_accuracy: 0.7583
Epoch 7/50
15/15 [==============================] - 0s 13ms/step - loss: 0.8075 - accuracy: 0.6500 - val_loss: 0.6971 - val_accuracy: 0.7583
Epoch 8/50
15

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 0.24340230226516724
Test accuracy: 0.8999999761581421
